In [1]:
import numpy as np
import pandas as pd

import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [2]:
import tensorflow as tf

In [3]:
#SPEI - 1 month, 6 month models (LSTM)
#SPEI with humidity, temperature models - 1 month (LSTM)
#ConvLSTM if time permits

In [4]:
#SPEI 6 month model

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
df = pd.read_csv("/content/drive/MyDrive/data_senegal/senegal_weather_spei_1970_2020_025x025_v3.csv")

In [18]:
df = df.dropna()
df = df.reset_index(drop=True)

In [19]:
spei_values = df["spei_6mon"]

In [10]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [20]:
n_steps = 12
X, y = split_sequence(spei_values, n_steps)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [27]:
scaler = StandardScaler()
X_train = scaler.fit_transform(np.array(X_train))
scaler = StandardScaler()
X_test = scaler.fit_transform(np.array(X_test))
scaler = StandardScaler()
y_train = scaler.fit_transform(y_train.reshape(-1,1))
scaler = StandardScaler()
y_test = scaler.fit_transform(y_test.reshape(-1,1))

In [28]:
n_features = 1 #Since it's a univariate model

model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [30]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

In [31]:
with tf.device('/device:GPU:0'):
  model.fit(X_train, y_train, epochs=60, batch_size=64) #25+28 epochs already - 0.040950603159884275 0.9570728210244714

Epoch 1/60
2309/2309 [==============================] - 97s 41ms/step - loss: 0.1651
Epoch 2/60
2309/2309 [==============================] - 95s 41ms/step - loss: 0.0905
Epoch 3/60
2309/2309 [==============================] - 94s 41ms/step - loss: 0.0841
Epoch 4/60
2309/2309 [==============================] - 94s 41ms/step - loss: 0.0763
Epoch 5/60
2309/2309 [==============================] - 95s 41ms/step - loss: 0.0743
Epoch 6/60
2309/2309 [==============================] - 95s 41ms/step - loss: 0.0705
Epoch 7/60
2309/2309 [==============================] - 95s 41ms/step - loss: 0.0658
Epoch 8/60
2309/2309 [==============================] - 96s 41ms/step - loss: 0.0632
Epoch 9/60
2309/2309 [==============================] - 95s 41ms/step - loss: 0.0597
Epoch 10/60
2309/2309 [==============================] - 95s 41ms/step - loss: 0.0569
Epoch 11/60
2309/2309 [==============================] - 95s 41ms/step - loss: 0.0529
Epoch 12/60
2309/2309 [==============================] - 95s 41

In [32]:
y_hat = model.predict(X_test)
print(mean_squared_error(y_hat, y_test))
print(sklearn.metrics.r2_score(y_hat, y_test))

0.019476872655452036
0.9802836379552837
